確認環境變數

In [ ]:
import os
import boto3
from dotenv import load_dotenv

# 載入 .env 文件中的環境變數
load_dotenv()

def check_credentials():
    # 初始化 boto3 session
    session = boto3.Session()
    print("Current Credentials:")
    print("Access Key:", session.get_credentials().access_key)
    print("Secret Key:", session.get_credentials().secret_key)
    print("Session Token:", session.get_credentials().token)
    print("Region:", session.region_name)

if __name__ == "__main__":
    check_credentials()


檢查使用者資訊

In [ ]:
import boto3

def get_account_id():
    try:
        # 初始化 STS 客戶端
        sts_client = boto3.client('sts')
        
        # 取得當前身份的資訊
        response = sts_client.get_caller_identity()
        
        # 輸出帳號 ID 和使用者 ARN
        print("Account ID:", response['Account'])
        print("User ARN:", response['Arn'])
    except Exception as e:
        print("無法取得帳號資訊，錯誤原因：", e)

if __name__ == "__main__":
    get_account_id()


查詢對話內容

In [ ]:
import boto3
from botocore.exceptions import ClientError

def fetch_and_display_records():
    # 初始化 DynamoDB 資源
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    table_name = "LinebotMessages"

    try:
        # 取得資料表
        table = dynamodb.Table(table_name)

        # 嘗試掃描資料表
        response = table.scan()
        items = response.get('Items', [])

        # 如果無資料，提示使用者
        if not items:
            print("當前無資料。")
        else:
            # 顯示所有記錄
            for item in items:
                print(
                    f"UserId: {item['UserId']}, \n"
                    f"Timestamp: {item['Timestamp']}, \n"
                    f"Message: {item['Message']}\n"
                )
    except ClientError as e:
        # 檢查是否是資料表不存在的錯誤
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"資料表 {table_name} 不存在。")
        else:
            print(f"讀取資料表時發生錯誤：{e}")
    except Exception as e:
        print(f"發生未知錯誤：{e}")

if __name__ == "__main__":
    fetch_and_display_records()
